In [1]:
import os
import mediapipe as mp
import cv2
import pickle
import matplotlib.pyplot as plt
from log import get_logger  # Import logger from our log.py
import logging
import numpy as np  # Added for sharpening kernel/array operations
import glob  # Added for efficient image counting

In [2]:
# Logger setup
logger = get_logger(__name__)
logging.getLogger('matplotlib.font_manager').disabled = True

In [ ]:
# Global constants
DATA_DIR = '/Users/ashishsingh/Desktop/SIGN01/data/numbers'
GRAPH_DIR = '/Users/ashishsingh/Desktop/SIGN01/graphs'
PROCESSED_DATA_DIR = '/Users/ashishsingh/Desktop/SIGN01/data/processed_data/numbers'

# Ensure graph directory exists
os.makedirs(GRAPH_DIR, exist_ok=True)

In [4]:
def init_mp_hands(static_mode=True, min_confidence=0.7, min_tracking_confidence=0.7):
    """
    Initializes and returns a MediaPipe Hands object.
    """
    logger.info("Initializing MediaPipe Hands...")
    mp_hands = mp.solutions.hands
    return mp_hands.Hands(
        static_image_mode=static_mode,
        min_detection_confidence=min_confidence,
        min_tracking_confidence=min_tracking_confidence
    )

In [5]:
def sharpen_image(image):
    """
    Applies unsharp masking to sharpen the image for better handling of blur.
    """
    blurred = cv2.GaussianBlur(image, (0, 0), sigmaX=3)
    sharpened = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    return sharpened

In [6]:
def count_total_images(data_dir):
    """
    Counts the total number of image files in the dataset directory and subdirectories.
    """
    image_paths = glob.glob(os.path.join(data_dir, '**/*.jpg'), recursive=True) + \
                  glob.glob(os.path.join(data_dir, '**/*.png'), recursive=True)  # Add other extensions if needed
    total = len(image_paths)
    logger.info(f"Found {total} images to process in {data_dir}.")
    return total

In [7]:
def process_images(data_dir, hands):
    """
    Processes images from the dataset directory using MediaPipe Hands
    and returns data and labels. Includes image sharpening for blur correction.
    """
    logger.info(f"Processing images from: {data_dir}")
    data = []
    labels = []

    # Count total images upfront
    total_images = count_total_images(data_dir)

    processed_count = 0  # To track images with successful detections
    skipped_count = 0    # Optional: Track skips for invalid loads or no detection

    for dir_ in os.listdir(data_dir):
        dir_path = os.path.join(data_dir, dir_)
        if dir_.startswith('.') or not os.path.isdir(dir_path):  # Skip hidden files and non-dirs
            logger.debug(f"Skipping: {dir_}")
            continue
        for img_filename in os.listdir(dir_path):
            img_path = os.path.join(dir_path, img_filename)
            if not os.path.isfile(img_path):  # Skip non-files
                continue

            img = cv2.imread(img_path)
            if img is None:
                logger.warning(f"Failed to load image: {img_path}")
                skipped_count += 1
                continue

            # Preprocess: Sharpen to handle potential blur
            sharpened_img = sharpen_image(img)

            # Convert to RGB for MediaPipe
            img_rgb = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2RGB)

            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                data_aux = [0] * 84  # 2 hands × (21 landmarks × 2 coords)
                for i, hand_landmarks in enumerate(results.multi_hand_landmarks[:2]):
                    x_ = [landmark.x for landmark in hand_landmarks.landmark]
                    y_ = [landmark.y for landmark in hand_landmarks.landmark]

                    hand_data = []
                    for landmark in hand_landmarks.landmark:
                        hand_data.append(landmark.x - min(x_))
                        hand_data.append(landmark.y - min(y_))

                    start_idx = i * 42
                    data_aux[start_idx:start_idx + 42] = hand_data

                data.append(data_aux)
                labels.append(dir_)
                processed_count += 1
            else:
                logger.debug(f"No hands detected in {img_path}")
                skipped_count += 1

    logger.info(f"Processing complete: {total_images} images total, MediaPipe detected hands in {processed_count} images (skipped {skipped_count}).")
    return data, labels

In [8]:
def save_data_pickle(data, labels, save_dir=PROCESSED_DATA_DIR):
    """
    Saves processed data and labels to a pickle file in processed_data folder.
    """
    os.makedirs(save_dir, exist_ok=True)  # Ensures directory exists
    save_path = os.path.join(save_dir, "data.pickle")
    with open(save_path, "wb") as f:
        pickle.dump({"data": data, "labels": labels}, f)
    logger.info(f"Processed data saved to {save_path}")

In [9]:
def visualize_data(labels, save_dir=GRAPH_DIR):
    """
    Creates a bar chart showing the number of samples per label.
    """
    logger.info("Generating dataset distribution visualization...")
    label_counts = {}
    for label in labels:
        label_counts[label] = label_counts.get(label, 0) + 1

    save_path = os.path.join(save_dir, 'data_visualization.png')

    plt.figure(figsize=(10, 6))
    plt.bar(label_counts.keys(), label_counts.values(), color='skyblue')
    plt.xlabel('Labels')
    plt.ylabel('Number of Samples')
    plt.title('Dataset Distribution')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

    logger.info(f"Visualization saved to {save_path}")

In [10]:
if __name__ == "__main__":
    # Initialize MediaPipe Hands
    hands = init_mp_hands()

    # Process images (with counting and sharpening integrated)
    data, labels = process_images(DATA_DIR, hands)

    # Save processed data
    save_data_pickle(data, labels)

    # Save visualization in /graphs
    visualize_data(labels)

2025-09-27 10:20:47,178 | INFO     | __main__ | Initializing MediaPipe Hands...
I0000 00:00:1758948647.255276   84161 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
2025-09-27 10:20:47,255 | INFO     | __main__ | Processing images from: /Users/ashishsingh/Desktop/SIGN/data/numbers
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1758948647.263858   84323 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758948647.269172   84324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-09-27 10:20:47,406 | INFO     | __main__ | Found 35275 images to process in /Users/ashishsingh/Desktop/SIGN/data/numbers.
2025-09-27 10:20:47,406 | DEBUG    | __main__ | Skipping: .DS_Store
W0000 00:00:1758948647.446600   84319 landmark_projection_calculator.cc:186]